# **Trabalho 3**

Todos os anos centenas de filmes são lançados no mundo todo por diversas produtoras que contam com orçamentos de todos os tamanhos, gerando uma indústria bilhonária. Nesse contexto, você como cientista da computação recém formado foi contratado para monitorar em tempo real as impressões da platéia por meio de posts em redes sociais.

Basicamente o seu trabalho é monitorar as impressões positivas e negativas de um determinado filme, nos últimos dias (exemplo: 30 dias). Você poderá fazer isso lendo todas as postagens e interpretando cada uma como positiva ou negativa ou **usando** o conhecimento adquirido durante a graduação e, portanto, automatizar essa tarefa. **Faça a escolha correta** e automatize essa tarefa. 

Para auxiliar nessa tarefa, considere o dataset anterior e os conceitos estudados em sala de aula.

Escolha três filmes* já lançados de sua preferência para conduzir essa análise. Lembre-se que seu trabalho é apresentar uma forma automática de entender impressões positivas e negativas sobre os filmes escolhidos a partir de posts em uma rede social (no caso, o Twitter).

O que você precisa apresentar é:

+ uma análise sobre os posts referentes aos filmes escolhidos (isto é, o notebbok contendo o código e comentado. Seja organizado). Por exemplo, espera-se que no mínimo você apresente um gráfico de impressões positivas e negativas sobre cada filme escolhido nos últimos **n** dias.

+ fazer um vídeo de no máximo 5 minutos explicando as tuas principais estratégias.

**Data de entrega:** 10 de agosto de 2022

*Seja prudente na escolha dos filmes, de modo que haja comentários suficientes para a sua análise.

## Dataset

In [ ]:
import pandas as pd
import io
import requests

In [ ]:
s = requests.get('https://raw.githubusercontent.com/ragero/text-collections/master/complete_texts_csvs/review_polarity.csv').content

dataset = pd.read_csv(io.StringIO(s.decode('utf-8')),
                na_values=['?'],
                skipinitialspace = True)
dataset = dataset.sample(frac=1)
dataset

,file_name,text,class
1641,cv281_23253.txt,"while watching boiler room , i was constantly ...",pos
1699,cv631_4967.txt,what surprises me most about the long-awaited ...,pos
1591,cv580_14064.txt,this movie was one of the first american films...,pos
236,cv338_9183.txt,wizards is an animated feature that begins wit...,neg
227,cv974_24303.txt,"long ago , films were constructed of strong di...",neg
...,...,...,...
1346,cv797_6957.txt,i have seen several ( but not that many ) wood...,pos
722,cv298_24487.txt,it's tough to be an aspiring superhero in cham...,neg
673,cv477_23530.txt,"the premise of the new james wong film , final...",neg
359,cv212_10054.txt,capsule : not as bad a sequel as crow 2 or bat...,neg


# Material suplementar

## BERT

In [ ]:
!pip install ktrain
import keras
import ktrain
from ktrain import text

## Pré-processando

In [ ]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(dataset,
                                                                      'text',
                                                                      label_columns='class',
                                                                      maxlen=64,
                                                                      max_features=50000,
                                                                      preprocess_mode='bert',
                                                                      lang=None,
                                                                      val_pct=0.1,
                                                                      )


In [ ]:
model = text.text_classifier('bert', (X_train, y_train) , preproc=preprocess)
classifier = ktrain.get_learner(model, 
                             train_data=(X_train, y_train), 
                             val_data=(X_test, y_test),
                             batch_size=64
                             )

Is Multi-Label? False
maxlen is 64
done.


In [ ]:
classifier.fit_onecycle(0.00002,1)



begin training using onecycle policy with max lr of 2e-05...
29/29 [==============================] - 1331s 45s/step - loss: 0.7208 - accuracy: 0.5083 - val_loss: 0.6839 - val_accuracy: 0.5500


In [ ]:
classifier.validate()

## Aplicando a novos reviews

In [ ]:
predictor = ktrain.get_predictor(classifier.model, preprocess)

In [ ]:
predictor.predict('The worst movie I have ever seen in my life.')

## Libraries para coletar posts do twitter
https://docs.tweepy.org/en/stable/index.html

In [ ]:
import tweepy
import textblob
import wordcloud

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re                         
plt.style.use('fivethirtyeight')

### Steps

 Para fazer a coleta você precisa ter uma conta de desenvolvedor do Twitter. Obter a conta de desenvolvedor do Twitter pode levar um ou dois dias (para criar sua conta [clique aqui](https://developer.twitter.com/en))

Neste [link](https://medium.com/@dprakash05/twitter-sentiment-analysis-f81f3d24c795) tem um tutorial de como fazer isso.

### Grabbing some tweets
Os próximos trechos de código foram obtidos deste [link](https://medium.com/@dprakash05/twitter-sentiment-analysis-f81f3d24c795)

### Setup account

In [ ]:
log=pd.read_csv('colocar_suas_credenciais')
consumerkey=log['key'][0]
consumersecret=log['key'][1]
accesstoken=log['key'][2]
accesstokensecret=log['key'][3]

In [ ]:
authenticate=tweepy.OAuthHandler(consumerkey,consumersecret)
## set access token and access token secret
authenticate.set_access_token(accesstoken, accesstokensecret)
## create API object while passing in authentication information
api=tweepy.API(authenticate, wait_on_rate_limit=True)

In [ ]:
tweets = tweepy.Cursor(api.search_30_day,
                              query='sea beast', 
                              #lang="en",
                              environment_name='prod').items(50)

In [ ]:
list_tweets = [tweet for tweet in tweets]
i = 1
for tweet in list_tweets[:15]:    
  print(str(i)+')'+tweet.text + '\n' )
  i+=1

## **Notebook útil e bem interessante que pode ajudar no trabalho**
https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_natural-language-processing-applications/sentiment-analysis-and-dataset.ipynb